#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [1]:
#names here

# EunJung Kim
# SuBin Lee

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [1]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [2]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [3]:
def get_data(filename):
    data = []
#     your code goes here

    df = pd.read_csv(filename, header=None)
    
    for index, row in df.iterrows():
        data_label = row[19]
        data_features = row[:19]

        dp = DataPoint(data_label, data_features)

        data.append(dp)
    return data

In [4]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [5]:
def make_prediction(tree_root, data_point):
#     your code goes here

    if(tree_root.is_leaf):
        return tree_root.prediction
    
    for data in data_point:
        if(data.features[tree_root.feature_idx] < tree_root.thresh_val):
            return make_prediction(tree_root.left_child, data_point)
        else:
            return make_prediction(tree_root.right_child, data_point)

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [6]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here

    for row in data:
        if row.features[feature_idx] < threshold:
            left_split.append(row)
        else:
            right_split.append(row)
            
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [7]:
from collections import Counter

def calc_entropy(data):
    entropy = 0.0
#     your code goes here

    labels = []
    c = len(data)    # number of classes
    
    for i in range(len(data)):
        labels.append(data[i].label)
        
    counted_labels = Counter(labels)    # fraction of records belonging to class
    
    for label in counted_labels.values():
        pi = label / c
        entropy += ((-pi) * log2(pi))

    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [8]:
def get_sorted_data(data, feature_idx):
    key = lambda x: x.features[feature_idx]
    return sorted(data, key=key)

In [9]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here

    sorted_data = get_sorted_data(data, feature_idx)
    parent_entropy = calc_entropy(sorted_data)

    thresh_list = []

    for i in range(len(data)-1):
        if(sorted_data[i].label != sorted_data[i+1].label):
            thresh_value = (sorted_data[i].features[feature_idx] + sorted_data[i+1].features[feature_idx]) / 2
            thresh_list.append(thresh_value)
    
    for thresh in thresh_list:
        left_split, right_split = split_dataset(sorted_data, feature_idx, thresh)
        
        left_entropy = calc_entropy(left_split)
        right_entropy = calc_entropy(right_split)
        
        split_entropy = ((len(left_split)/len(data))*left_entropy) + ((len(right_split)/len(data))*right_entropy)
        gain = parent_entropy - split_entropy
        
        if(best_info_gain < gain):
            best_info_gain = gain
            best_thresh = thresh
        
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [10]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here

    best_info_gain = 0.0

    for feature_idx in range(len(data[0].features)):
        feature_info_gain, feature_thresh = calc_best_threshold(data, feature_idx)
        
        if feature_info_gain > best_info_gain:
            best_info_gain = feature_info_gain
            best_thresh = feature_thresh
            best_feature = feature_idx
    
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [11]:
def create_leaf_node(data):
    labels = [d.label for d in data]
    counts = Counter(labels)
    most_common_label = counts.most_common(1)[0][0]    # Get label that classification occurs most

    leaf_node = TreeNode()
    leaf_node.is_leaf = True
    leaf_node.prediction = most_common_label
    
    return leaf_node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [12]:
def create_decision_tree(data, max_levels):
#     your code goes here

    if(max_levels == 1):
        return create_leaf_node(data)
    
    best_feature, best_thresh = identify_best_split(data)
    if(best_feature is None):
        return create_leaf_node(data)
    
    left_split, right_split = split_dataset(data, best_feature, best_thresh)
    
    node = TreeNode()
    node.is_leaf = False
    node.feature_idx = best_feature
    node.thresh_val = best_thresh
    node.left_child = create_decision_tree(left_split, max_levels - 1)
    node.right_child = create_decision_tree(right_split, max_levels - 1)
    
    return node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [24]:
def calc_accuracy(tree_root, data):
#     your code goes here
    correct = 0
    
    for data_point in data:
        prediction = make_prediction(tree_root, [data_point])
        
        if prediction == data_point.label:
            correct += 1
        
    accuracy = correct / len(data)

    return accuracy

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [42]:
import random

def split_train_test(data, ratio):
    n = len(data)
    n_test = int(n * ratio)
    
    test_indices = random.sample(range(n), n_test)
    train_indices = [i for i in range(n) if i not in test_indices]
    
    return [data[i] for i in train_indices], [data[i] for i in test_indices]

In [44]:
# Q10
import time

d = get_data("messidor_features.txt")

accuracy = 0
total_acc = 0
k = 5

for i in range(k):
    start = time.time()
    
    print("Test ", i)

    # partition data into train_set and test_set
    train_set, test_set = split_train_test(d, 0.2)

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    
    end = time.time()
    print ('Time taken:', end - start)
    
    total_acc += accuracy
    
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    
# average the accuracy over k folds
total_acc = total_acc / k
print ('The average accuracy is ', str(total_acc * 100.0))
# tree.printTree()

Test  0
Training set size: 920
Test set size    : 230
Time taken: 54.87875509262085
The accuracy on the test set is  64.34782608695652
Test  1
Training set size: 920
Test set size    : 230
Time taken: 67.29963088035583
The accuracy on the test set is  60.86956521739131
Test  2
Training set size: 920
Test set size    : 230
Time taken: 53.162062883377075
The accuracy on the test set is  64.34782608695652
Test  3
Training set size: 920
Test set size    : 230
Time taken: 56.871042013168335
The accuracy on the test set is  68.69565217391305
Test  4
Training set size: 920
Test set size    : 230
Time taken: 60.08712077140808
The accuracy on the test set is  63.91304347826087
The average accuracy is  64.43478260869566


In [ ]:
###############################

In [150]:
# TEST CODE

## FULL DATASET
print('\nFULL DATASET')
full_data = get_data("messidor_features.txt")
print(f'Length of dataset: {len(full_data)}')

print(f'Entropy of dataset: {calc_entropy(full_data)}')

best_gain, best_thresh = calc_best_threshold(full_data, 3)
print("Best thresh:", best_thresh)
print("Best gain:", best_gain)

feat, thresh = identify_best_split(full_data)
print(f"Best split: feature_index={feat}, thresh={thresh}")


FULL DATASET
Length of dataset: 1150
Entropy of dataset: 0.9971705766292643
Best thresh: 62.5
Best gain: 0.04595772209113502
Best split: feature_index=14, thresh=0.0203705


In [208]:
temp_data = get_data("sample_train.txt")

print("Create Tree:")
temp_tree = create_decision_tree(temp_data, 10)
temp_tree.printTree()

Create Tree:
Internal Node:  splits on feature 6 with threshold 25.5
-Internal Node:  splits on feature 1 with threshold 0.5
--Leaf Node:      predicts 1.0
--Internal Node:  splits on feature 8 with threshold 30.5556585
---Internal Node:  splits on feature 2 with threshold 21.5
----Leaf Node:      predicts 1.0
----Leaf Node:      predicts 0.0
---Leaf Node:      predicts 0.0
-Internal Node:  splits on feature 16 with threshold 0.487661
--Leaf Node:      predicts 0.0
--Leaf Node:      predicts 1.0
